In [32]:
import openai

In [33]:
from dotenv import dotenv_values

In [34]:
config = dotenv_values('.env')

In [35]:
openai.api_key=config['OPENAI_API_KEY']

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('sales_data_sample.csv', encoding='cp437')

In [8]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [9]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [10]:
import sqlalchemy

In [13]:
temporary_db = create_engine('sqlite:///:memory:',echo=True)

In [16]:
data = df.to_sql('Sales',con=temporary_db)

2023-07-19 10:59:14,259 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-07-19 10:59:14,260 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-19 10:59:14,261 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-07-19 10:59:14,262 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-19 10:59:14,270 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-19 10:59:14,275 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [17]:
with temporary_db.connect() as conn:
    result=conn.execute(text('SELECT * from Sales'))

2023-07-19 10:59:31,115 INFO sqlalchemy.engine.Engine SELECT * from Sales
2023-07-19 10:59:31,116 INFO sqlalchemy.engine.Engine [cached since 194.1s ago] ()


In [21]:
def create_definition(db):
    prompt="""
    #Sales({})
    """.format(','.join(str(col)for col in result.keys()))
    return prompt

In [22]:
create_definition(temporary_db)

'\n    #Sales(index,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    '

In [36]:
def prompt_input():
    nlp_text = input('What do you want to do?: ')
    return nlp_text

In [37]:
def combine(df,prompt_input):
    definition = create_definition(df)
    query=f'Query to answer: {prompt_input()}'
    return definition+query

In [38]:
response=openai.Completion.create(
model='text-davinci-003',
prompt=combine(df,prompt_input),
temperature=0,
max_tokens=150,
frequency_penalty=0,
presence_penalty=0,
stop=['#',';'],
n=1)

What do you want to do?: sum of sales per state


In [39]:
response

<OpenAIObject text_completion id=cmpl-7dxNfWqmN1zAm5Zdwcmw1Z3cl49hk at 0x2a6dbb0be00> JSON: {
  "id": "cmpl-7dxNfWqmN1zAm5Zdwcmw1Z3cl49hk",
  "object": "text_completion",
  "created": 1689757407,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n    SELECT STATE, SUM(SALES)\n    FROM Sales\n    GROUP BY STATE",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 138,
    "completion_tokens": 20,
    "total_tokens": 158
  }
}

In [48]:
def return_sql_query(response):
    query=response['choices'][0]['text']
    if query.startswith(' '):
        query='SELECT'+query
    return query

In [49]:
return_sql_query(response)

'\n    SELECT STATE, SUM(SALES)\n    FROM Sales\n    GROUP BY STATE'

In [50]:
with temporary_db.connect() as conn:
    result = conn.execute(text(return_sql_query(response)))

2023-07-19 11:04:22,941 INFO sqlalchemy.engine.Engine 
    SELECT STATE, SUM(SALES)
    FROM Sales
    GROUP BY STATE
2023-07-19 11:04:22,942 INFO sqlalchemy.engine.Engine [cached since 18.52s ago] ()


In [51]:
result.all()

[(None, 5283535.709999999),
 ('BC', 149873.77000000002),
 ('CA', 1505542.229999999),
 ('CT', 238661.28000000006),
 ('Isle of Wight', 78240.83999999998),
 ('MA', 666443.7199999996),
 ('NH', 131685.30000000002),
 ('NJ', 83228.19),
 ('NSW', 305567.1099999999),
 ('NV', 82751.08000000002),
 ('NY', 646343.7599999998),
 ('Osaka', 67605.07),
 ('PA', 273327.26999999996),
 ('Quebec', 74204.79),
 ('Queensland', 59469.11999999999),
 ('Tokyo', 120562.73999999996),
 ('Victoria', 265586.86999999994)]